In [116]:
%run ibm_connection.py
# Import necessary libraries
from qiskit import QuantumCircuit, Aer, execute
import numpy as np
from qiskit.circuit.library import *
from fractions import Fraction
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from math import gcd # greatest common divisor
from math import log, e, floor
import time

def isprime(num):
    '''
        Checks if a number is prime
        Input: num(integer)
        Output: True or False
    '''
    for n in range(2,int(num**0.5)+1):
        if num%n==0:
            return False
    return True

def is_coprime(a, N):
    '''
        Checks if an integer a is coprime to another (N)
        Input: a(int), N(int)
        Output: True or False
    '''
    for i in range(2, a+1):
        if a%i==0 and N%i==0:
            return False
    return True

def shor_circuit(N, a, t):
    '''
        Constructs the Shor circuit
        Input:  N(int), the number to be factored
                a(int), coprime of N
                t, number of counting qubits
        Output: qc, the quantum circuit of t+1 qubits and t classical bits
    '''
    # Define the quantum circuit with t+1 qubits and t-bit classical register
    qc = QuantumCircuit(t+1, t)

    # Apply Hadamard gates to the first 6 qubits
    qc.h(range(t))
    qc.x(t)

    # Apply a phase estimation circuit to estimate the eigenvalues of the unitary operator Ua = a^x mod N
    for counting_qubit in range(t):
        gate = QuantumCircuit(1)
        gate.u(0, (log((a**(2**counting_qubit))%N)), 0, 0);
        gate = gate.to_gate()
        gate = gate.control()
        qc.append(gate, [counting_qubit, t])
        qc.barrier()

    # Do the inverse QFT:
    qc = qc.compose(QFT(t, inverse=True), range(t))
    qc.barrier()

    # Measure the first t qubits and store the measurement result in the classical register cr1
    qc.measure(range(t), range(t))
    return qc

#---------------------------------------------------------------------------------------------------------------
# Define the number to be factored
N = input("Type integer to be factored:")

#Accepts two primes multiplied, e.g. 3*7
if "*" in N:
    split = N.split("*")
    N = int(split[0])
    for i in split[1:]:
        N = N * int(i)
    print("N = ", N)
else:
    N = int(N)
    
#Define a
max_a = 8
for i in range(max_a):
    a = max_a - i
    if is_coprime(a, N):
        break
if not is_coprime(a, N):
    a = a + 1
    while True:
        if is_coprime(a, N):
            break
        else:
            a = a + 1
print("a =", a)

'''
#Uncomment to use user-input a

#Define a
while True:
    a = int(input("Type a coprime integer to be used as a: "))
    if is_coprime(a,N):
        break
    else:
        print("The number you typed is not coprime to N.")
'''

#Define t
t = 2*N.bit_length() + 2

limit = 24 #Change based on the computer's abilities
if t+1>limit:
    print("The number of qubits needed is %d + 1, which more than the current limit of %d." %(t, limit))
    user_limit = input("Set new limit (Press enter to ignore):")
    if user_limit and t+1>int(user_limit):
        limit = int(user_limit)
    print("Circuit will only use %d qubits" %(limit))
    t = limit - 1
else:
    print("Qubits that are going to be used: ", t+1)

#Construct circuit
t0 = time.time()
qc = shor_circuit(N,a,t)

#Running circuit and classical post-processing
attempt = 0
factor_found = False
while not factor_found:
    attempt += 1
    # Run the circuit on a simulator
    backend = Aer.get_backend('qasm_simulator')
    counts = execute(qc, backend, shots=1024).result().get_counts()

    # Analyze the measurement results to obtain the phase
    reading = max(counts, key=lambda x: counts[x])
    phase = int(reading, 2) / 2**t
    frac = Fraction(phase).limit_denominator(N)  # Find the fraction approximation of the phase
    r = frac.denominator  # Obtain the denominator as the potential period

    if phase != 0:
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0 and isprime(guess) and isprime(N/guess): # Check to see if guess is a factor
                print("*** Non-trivial factor found: %i ***" % guess)
                factor_found = guess
                
    #Do the same for the second most probable outcome
    if not factor_found:
        counts.pop(reading)
        if len(counts):
            reading = max(counts, key=lambda x: counts[x])
            phase = int(reading, 2) / 2**t
            frac = Fraction(phase).limit_denominator(N)  # Find the fraction approximation of the phase
            r = frac.denominator  # Obtain the denominator as the potential period

            if phase != 0:
                guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
                for guess in guesses:
                    if guess not in [1,N] and (N % guess) == 0 and isprime(guess) and isprime(N/guess): # Check to see if guess is a factor
                        print("*** Non-trivial factor found: %i ***" % guess)
                        factor_found = guess
    if factor_found:
        print("\n\n%d = %d * %d" %(N, factor_found, N/factor_found))
        print("Total attempts:", attempt)
        print("Time:", time.time() - t0, "seconds")
        
    #Time-out after 50 attempts
    if attempt == 50:
        print("Maximum attempts reached. Prime factors not found.")
        new_a = input("Choose a different a to try again (Enter to ignore): ")
        if new_a:
            print("Restarting...")
            attempt = 0
            a = int(new_a)
            qc = shor_circuit(N,a,t)
        else:
            print("\nMore control qubits are needed.")
            print("Time:", time.time() - t0, "seconds")
            break

Type integer to be factored:89*97
N =  8633
a = 8
The number of qubits needed is 30 + 1, which more than the current limit of 24.
Set new limit (Press enter to ignore):
Circuit will only use 24 qubits
*** Non-trivial factor found: 89 ***


8633 = 89 * 97
Total attempts: 1
Time: 2.358499765396118 seconds


In [49]:
N=979
print(2*N.bit_length())

20
